In [1]:
import pandas as pd
import numpy as np
import random
from sklearn import datasets
from multiprocessing.dummy import Pool

In [2]:
#df = pd.read_csv('covid.csv')
#global df

In [3]:
covid = pd.read_csv('covid_test.csv')

C:\Users\malu.maia\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (5,6,7,8,9,15,20,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,50,51,52,53,54,55,56,58,59,60,61) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
covid = covid[['estadoCaso', 'municipioCaso', 'bairroCaso', 'sexoCaso', 'dataNascimento', 
               'idadeCaso','racaCor', 'resultadoFinalExame']]

df = covid

# Suppression

In [5]:
def suppression_value(feature, value=''):
    df_s = df.copy()
    if(value == ''):
        for line in range(len(df_s)):
            df_s[feature].iloc[line] = '*'
    else:
        df_s = df_s.replace(value, '*')
    return df_s

In [6]:
def suppression(features):
    df_s = df.copy()
    for feature in features:
        values = np.unique(df[feature])
        dict_rep = {key: '*' for key in values}
        df_s[feature] = df_s[feature].replace(dict_rep)
    return df_s

### Testing Suppression on COVID data

In [7]:
suppression(['racaCor']).head()

,estadoCaso,municipioCaso,bairroCaso,sexoCaso,dataNascimento,idadeCaso,racaCor,resultadoFinalExame
0,CE,SOBRAL,CAICARA,MASCULINO,2003-08-14,17.0,*,Negativo
1,CE,PACAJUS,BANGUE,MASCULINO,1983-11-07,37.0,*,Negativo
2,CE,FORTALEZA,ZUMBI,MASCULINO,1992-03-12,28.0,*,Negativo
3,CE,CAUCAIA,METROPOLIS,MASCULINO,1970-03-06,50.0,*,Negativo
4,CE,FORTALEZA,PARANGABA,FEMININO,1939-01-10,81.0,*,Negativo


In [8]:
suppression_value('racaCor', 'Parda').head()

,estadoCaso,municipioCaso,bairroCaso,sexoCaso,dataNascimento,idadeCaso,racaCor,resultadoFinalExame
0,CE,SOBRAL,CAICARA,MASCULINO,2003-08-14,17.0,*,Negativo
1,CE,PACAJUS,BANGUE,MASCULINO,1983-11-07,37.0,*,Negativo
2,CE,FORTALEZA,ZUMBI,MASCULINO,1992-03-12,28.0,*,Negativo
3,CE,CAUCAIA,METROPOLIS,MASCULINO,1970-03-06,50.0,Sem Informacao,Negativo
4,CE,FORTALEZA,PARANGABA,FEMININO,1939-01-10,81.0,*,Negativo


# Generalization

In [ ]:
def generalization():
    return

### Generalizing County

### Generalizing birthday

# Perturbation

In [9]:
from scipy.stats import laplace

In [10]:
def adding_laplace_noise(index, value):
    noise = laplace.rvs(loc=0, scale = sensitivity/eps)
    value = np.ceil(round(value + noise, 1)) # remove ceil(*) for float values
   
    return index, value

In [11]:
def perturbation(feature):
    global sensitivity
    global eps
    df_copy = df.copy()[:20000]
    if(isinstance(df_copy[feature].iloc[0], str)):
        values, counts = df_copy[feature].value_counts().index, df_copy[feature].value_counts().values
        df_copy[feature] = random.choices(values, weights=counts, k=len(df_copy[feature])) # following the real frequencies distribution
    else:
        sensitivity = max(df_copy[feature]) - min(df_copy[feature])
        eps = 10
        index_feature_2D = np.c_[np.array(df_copy.index), np.array(df_copy[feature])]
        
        with Pool(15) as pool:
            threads = pool.starmap(adding_laplace_noise, index_feature_2D)
            pool.terminate()
            pool.close()
        pool.join()
        
        df_copy[feature] = [value for value in list(dict(threads).values())]
    
    return df_copy

### Testing Perturbation on COVID data

In [12]:
feature = 'idadeCaso'
df_p = perturbation(feature)
df_p

,estadoCaso,municipioCaso,bairroCaso,sexoCaso,dataNascimento,idadeCaso,racaCor,resultadoFinalExame
0,CE,SOBRAL,CAICARA,MASCULINO,2003-08-14,45.0,Parda,Negativo
1,CE,PACAJUS,BANGUE,MASCULINO,1983-11-07,32.0,Parda,Negativo
2,CE,FORTALEZA,ZUMBI,MASCULINO,1992-03-12,28.0,Parda,Negativo
3,CE,CAUCAIA,METROPOLIS,MASCULINO,1970-03-06,41.0,Sem Informacao,Negativo
4,CE,FORTALEZA,PARANGABA,FEMININO,1939-01-10,73.0,Parda,Negativo
...,...,...,...,...,...,...,...,...
19995,CE,ACARAPE,SAO FRANCISCO,FEMININO,1979-01-03,31.0,Branca,Negativo
19996,CE,RERIUTABA,CAMPO LINDO,FEMININO,2000-03-07,35.0,Amarela,Positivo
19997,CE,IBIAPINA,ZONA RURAL,FEMININO,1986-12-01,30.0,Branca,Positivo
19998,CE,ACARAPE,SAO BENEDITO,FEMININO,1952-04-06,77.0,Parda,Positivo


In [14]:
print('Médias perturbada e não-perturbada: \n- {}\n- {}\n'.format(np.mean(df_p[feature]), 
                                                                np.mean(df[feature].iloc[:20000])))
print('Medianas perturbada e não-perturbada: \n- {}\n- {}\n'.format(np.median(df_p[feature]), 
                                                                  np.median(df[feature].iloc[:20000])))

Médias perturbada e não-perturbada: 
- 44.36855
- 43.9545

Medianas perturbada e não-perturbada: 
- 43.0
- 41.0

